In [ ]:
## !/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
import numpy as np
import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn


import sys
import itertools

#from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "16g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

In [2]:
import numpy as np
t = sc.parallelize(np.random.rand(20,4))

Модель
$$\hat{m}_{ij} = 1/(1+e^{-s_{ij}})$$
$$s_{ij} = \mu + a_i + b_j + \sum_k X_{ik} \cdot Y_{kj}$$
$$L_{ij} = -m_{ij} \cdot \ln(\hat{m}_{ij}) - (1 - m_{ij}) \cdot \ln(1 - \hat{m}_{ij}) + \lambda_{0} \cdot \mu + \lambda_{1}\cdot (a_i^2 + b_j^2) + \lambda_{2} \cdot \sum_k(X_{ik}^2 + Y_{kj}^2)$$

In [1]:
import datetime
start_time = datetime.datetime.now()
# Param selection
query = '''
select id_num,urlfr_num from 
(select id_num,urlfr_num,count(*) over (partition by id_num) as uf_cnt,count(*) over (partition by urlfr_num) as id_cnt from user_kposminin.visits_enum_20160412 
where id_num < 10000) a
where id_cnt > 10 and uf_cnt > 9
'''
# where uf_cnt > 9 and id_cnt > 500


K = 50
id_cnt = 10000
urlfr_cnt = 3000000
epochs = 15


#lmbd = np.array([0.01, 0.01, 0.01])

#step = np.array([0.001, 0.005, 0.01])

param_grid = [[[0.00497871,0.04978707,0.00497871],[4.53999E-07,4.53999E-06,4.53999E-05]]]

 #   [[0.00001, 0.0001, 0.0001],[0.01, 0.05, 0.01]],
for base_step in np.exp(np.arange(-8,-2,1)):
    for base_lmbd in np.exp(np.arange(-10,0,2)):
        for var_step in np.array([[0.1,1,0.1],[0.1,0.1,1],[0.01,0.1,1]]):
            for var_lmbd in np.array([[0.01,0.1,1],[1,0.1,0.01],[1,1,1]]):
                param_grid.append([var_step * base_step,var_lmbd * base_lmbd])
'''
# Load and parse the data
sampled_data = hc.sql(query) \
            .collect()
print('Sampled data consists of {} rows, {} id and {} uf.'.format(len(sampled_data),len(set([e[0] for e in sampled_data])),len(set([e[1] for e in sampled_data]))))

train,test = [], []
for r in sampled_data:
    if(np.random.rand() < 0.1):
        test.append(r)
    else:
        train.append(r)
'''     
# 
# prediction is m^hat_{ij} = 1/(1+exp(-s_{ij})) where s_{ij} = mu + a[i] + b[j] + sum_k (X[i,k] * Y[k,j])
# error function is L_{ij} = -m_{ij}*ln(m^hat_{ij}) - (1 - m_{ij})*ln(1 - m^hat_{ij}) + 
#                            lmbd[0] * mu + lmbd[1]*(a_i^2 + b_j^2) + lmbd[2]*sum_k(X_{ik}^2 + Y_{kj}^2)
f = open('data/collab_filter_SGD_param_tuning.csv','a+')
stat = []
prev_id = -1
uf_visited= []
neg_sig_share = 2

for step,lmbd in param_grid:    
    #Init matrices
    mu = np.random.rand()
    a = np.random.rand(id_cnt)
    b = np.random.rand(urlfr_cnt)
    X = np.random.rand(id_cnt, K) / (K ** 0.5)
    Y = np.random.rand(K,urlfr_cnt) / (K ** 0.5)
    
    def make_step(i, j, v):
        s = mu + a[i] + b[j] + X[i,:].dot(Y[:,j])
        pred = 1./(1+np.exp(-s))
        err = v - pred # r[2] - pred
        grad_mu = -err + lmbd[0] * mu
        grad_ai = -err + lmbd[1] * a[i]
        grad_bj = -err + lmbd[1] * b[j]
        grad_xi = -err * Y[:,j] + lmbd[2] * X[i,:]
        grad_yj = -err * X[i,:] + lmbd[2] * Y[:,j]
        # TODO Implement Adadelta SGD version. 
        #http://int8.io/comparison-of-optimization-techniques-stochastic-gradient-descent-momentum-adagrad-and-adadelta/#AdaDelta_8211_implementation
        mu   += - grad_mu * step[0]
        a[i] += - grad_ai * step[1]
        b[j] += - grad_bj * step[1]
        X[i,:] = X[i,:] - grad_xi * step[2]
        Y[:,j] = Y[:,j] - grad_yj * step[2]

    for epoch in range(epochs):
        for r in train:
            i,j = r[:2]
            make_step(i,j,1)
            if i == prev_id: 
                uf_visited.append(j)
            else:
                # Add negative examples
                ni = np.random.randint(len(train) - len(uf_visited) - 5)
                n = 0
                while(n < int(len(uf_visited) * neg_sig_share)):
                    if(not train[ni + n][1] in uf_visited):
                        make_step(prev_id, train[ni + n][1],0)
                        n +=1                
                uf_visited = []
                prev_id = i

        
        #calc err
        err1 ,err2 = 0, 0
        for r in test:
            i,j = r[:2]
            s = mu + a[i] + b[j] + X[i,:].dot(Y[:,j])
            pred = 1/(1+np.exp(-s))
            err1 += - 1 * np.log(pred) # r[2] - pred
            err2 += - 1 * np.log(pred) + lmbd[0] * mu + lmbd[1] * (a[i] ** 2 + b[j] ** 2) + lmbd[2] * (X[i,:].dot(X[i,:]) + Y[:,j].dot(Y[:,j]))
        err1 /= float(len(test))
        err2 /= float(len(test))
        
        stat=('{} ' * 5).format(step,lmbd,epoch,err1, err2)
        #f.write(stat + '\n')
        print(stat)
        
#open('data/collab_filter_SGD_param_tuning.csv','a+').write(str(stat))
print('Finish. Work time {}.'.format(datetime.datetime.now()- start_time))
#f.write('\nFinish at {}. Work time {}.\n\n\n'.format(datetime.datetime.now(),datetime.datetime.now() - start_time))
f.close()

NameError: name 'np' is not defined

1.00000453999

In [111]:
#from IPython.display import display, Math, Latex
#display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))
display(Math('$$\hat{m_{ij}} = 1/(1+e^{-s_{ij}}), s_{ij} = \mu + a_i + b_j + \sum_k X_{ik} * Y_{kj}$$'))
# error function is L_{ij} = -m_{ij}*ln(m^hat_{ij}) - (1 - m_{ij})*ln(1 - m^hat_{ij}) + 
#                            lmbd[0] * mu + lmbd[1]*(a_i^2 + b_j^2) + lmbd[2]*sum_k(X_{ik}^2 + Y_{kj}^2)


<IPython.core.display.Math object>

In [114]:
err1 ,err2 = 0, 0
for k in range(100):
    if 1==1:
        if 1==1:
            i,j = test[k][0],test[i][1]
            s = mu + a[i] + b[j] + X[i,:].dot(Y[:,j])
            pred = 1/(1+np.exp(-s))
            err1 += - 1 * np.log( pred) # r[2] - pred
            err2 += - 1 * np.log( pred) + lmbd[0] * mu + lmbd[1] * (a[i] ** 2 + b[j] ** 2) + lmbd[2] * (X[i,:].dot(X[i,:]) + Y[:,j].dot(Y[:,j]))
err1 /= 100.0
err1, err2/100.

(0.0089550880129666641, 0.01349197871646092)